# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

In [288]:
import csv
import pprint


def pprint_dict_of_lists(dct, m=3):
    pprint.pprint({k: [*v[:m], '...', *v[-m:]] for k, v in list(dct.items())})


with open("./data/open_pubs.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)
    data = {header: [] for header in headers}

    for row in reader:
        for i in range(len(headers)):
            data[headers[i]].append(row[i])

# pprint_dict_of_lists(data)  # реально круто!


2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [289]:
import numpy as np
from timeit import default_timer

start = default_timer()
size = (2000, 2000)
A = np.random.randint(0, 100, size=size, dtype=np.int8)
B = np.random.randint(0, 100, size=size, dtype=np.int8)
C = np.dot(A, B)

print(f'time: {(default_timer() - start) * 1000:.4f} ms')

np.savez("./data/out/result.npz", A, B, C)
# time: 8649.8260 ms

time: 8350.0326 ms


3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

In [290]:
import h5py
A = np.random.uniform(0, 100, size=(1000, 1000)) # непрерывное равномерное распределение
B = np.random.binomial(100, 0.5, size=(1000, 1000))

with h5py.File("./data/out/test.h5", "w") as hdf:
    ds1 = hdf.create_dataset("arrA", data=A)
    ds2 = hdf.create_dataset("arrB", data=B)

    ds1.attrs["Description"] = "Array A with uniform distribution"
    ds1.attrs["low"] = 0
    ds1.attrs["high"] = 100
    ds2.attrs["Description"] = "Array B with binomial distribution"
    ds2.attrs["low"] = 100
    ds2.attrs["high"] = 0.5


In [291]:
with h5py.File("./data/out/test.h5", "r") as hdf:
    for k in hdf.keys():
        ds = hdf[k]
        print(ds.attrs['Description'])

Array A with uniform distribution
Array B with binomial distribution


## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [292]:
import csv
import json

with open("./data/tags_sample.csv", "r") as f:
    reader = csv.reader(f)
    headers = next(reader)
    tags_data = {}

    for row in reader:
        if int(row[0]) not in tags_data:
            tags_data[int(row[0])] = []
        tags_data[int(row[0])].append(row[1])


with open("./data/out/tags_sample.json", 'w') as f:
    json.dump(tags_data, f)

# pprint_dict_of_lists(tags_data)

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [293]:
import pandas as pd
df = pd.read_csv("./data/recipes_sample_with_filled_nsteps.csv", sep=",")

df["n_tags"] = [len(v) for v in tags_data.values()]
df["tags"] = [";".join(v) for v in tags_data.values()]

# print(df)

1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

In [294]:
with open("./data/ingredients_sample.csv", "r") as f:
    reader = csv.DictReader(f)
    ingredient_data = {}

    for row in reader:
        idd = row["recipe_id"]
        if not idd in ingredient_data:
            ingredient_data[idd] = []
        ingredient_data[idd].append(row["ingredient"])

# print(ingredient_data)


1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

In [296]:
df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,18.0,25,weeknight;time-to-make;course;main-ingredient;...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,NaN,31,15-minutes-or-less;time-to-make;course;prepara...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8.0,17,30-minutes-or-less;time-to-make;course;main-in...
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,NaN,11,60-minutes-or-less;time-to-make;course;prepara...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,NaN,19,30-minutes-or-less;time-to-make;course;main-in...
...,...,...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16,this is based on a french recipe but i changed...,10.0,18,time-to-make;course;main-ingredient;cuisine;pr...
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22,"this is a traditional fresh plum cake, thought...",11.0,19,time-to-make;course;main-ingredient;cuisine;pr...
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10,this is a traditional late summer early fall s...,NaN,20,time-to-make;course;main-ingredient;cuisine;pr...
29998,zydeco soup,486161,60,227978,2012-08-29,7,this is a delicious soup that i originally fou...,NaN,20,ham;60-minutes-or-less;time-to-make;course;mai...


In [297]:
# ингридиенты из csv файла
df["n_ingredients_csv"] = [len(v) for v in ingredient_data.values()]
mask = df['n_ingredients'].isna()
df.loc[mask, "n_ingredients"] = df.loc[mask, "n_ingredients_csv"]  # заменяем на значения из csv
df = df.drop(columns=['n_ingredients_csv'])
df


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,18.0,25,weeknight;time-to-make;course;main-ingredient;...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,6.0,31,15-minutes-or-less;time-to-make;course;prepara...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8.0,17,30-minutes-or-less;time-to-make;course;main-in...
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,8.0,11,60-minutes-or-less;time-to-make;course;prepara...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,4.0,19,30-minutes-or-less;time-to-make;course;main-in...
...,...,...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16,this is based on a french recipe but i changed...,10.0,18,time-to-make;course;main-ingredient;cuisine;pr...
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22,"this is a traditional fresh plum cake, thought...",11.0,19,time-to-make;course;main-ingredient;cuisine;pr...
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10,this is a traditional late summer early fall s...,2.0,20,time-to-make;course;main-ingredient;cuisine;pr...
29998,zydeco soup,486161,60,227978,2012-08-29,7,this is a delicious soup that i originally fou...,2.0,20,ham;60-minutes-or-less;time-to-make;course;mai...


1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

In [298]:
import numpy as np
if df["n_ingredients"].isna().sum() == 0:
    df["n_ingredients"] = df["n_ingredients"].astype(np.int)
    df.to_csv("./data/out/recipes_sample_with_tags_ingredients.csv", index=False)


### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.